In [1]:
# %matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
#from matplotlib import animation

In [2]:
class Parabola():
    def __init__(self, a, vertex_x=0, vertex_y=0):
        self.a = a  
        self.prev = 1
        
    def get_abscissa(self, y):
        return y**2 / (4*self.a)
        
    def draw_locus(self, x):
        return np.append(x[::-1], x), np.append((2 * np.sqrt(self.a * x))[::-1], -2 * np.sqrt(self.a * x))
    
    def draw_tangent(self, at_point_y):
        at_point_x = at_point_y**2 / (4*self.a)
        x = np.linspace(at_point_x-1, at_point_x+1, 5)
        return x, [(2*self.a/at_point_y) * (i - at_point_x) + at_point_y if at_point_y != 0 else 0 for i in x]
    
    def draw_normal(self, at_point_y):
        at_point_x = at_point_y**2 / (4*self.a)
        x = np.linspace(at_point_x-1, at_point_x+1, 5)
        return x, (at_point_y / (2*self.a)) * (at_point_x - x) + at_point_y
    
    def get_reflected_ordinate(self, x, x_incident, y_incident):
        if self.a == x_incident:
            if self.prev > 0:
                self.prev = -1
                return x_incident, y_incident
            else:
                self.prev = 1
                return x_incident, -1*y_incident - 1
                        
        elif x_incident < self.a:
            return x, (4*self.a*y_incident * (x_incident - x)) / (4*self.a**2 - y_incident**2) + y_incident
        else:
            if x == x_incident:
                return x, (4*self.a*y_incident * (x_incident - x)) / (4*self.a**2 - y_incident**2) + y_incident
            return self.a/2, ((4*self.a*y_incident * (x_incident - self.a/2)) / (4*self.a**2 - y_incident**2) + y_incident)

In [3]:
class ParallelIncidentLight():
    def __init__(self, y, reflector):
        self.y = y
        self.reflector = reflector
        self.x_incident = reflector.get_abscissa(y)
        self._is_reflected = True
    
    @property
    def is_reflected(self):
        return bool(self._is_reflected)
    
    @is_reflected.setter
    def is_reflected(self, value):
        
        if value == True or value == False:
            self._is_reflected = value 
        elif not isinstance(value, (bool, int, float)):
            raise TypeError (f"Invalid type for value. Expected any of type bool, int, or float but got {str(type(value))[8:-2]}")
        else:
            raise ValueError (f"Invalid value. Expected any of type bool, 1, or 0 but got {value}")
            
    def get_ordinate(self, x):
        if self.is_reflected:
            return self.reflector.get_reflected_ordinate(x, self.x_incident, self.y)
        else:
            return x, self.y

In [4]:
def update_plot(q = 3, a = 4):
    fig = plt.figure(figsize=(5,5))
    i_init=20
    
    par = Parabola(a)

    i = ParallelIncidentLight(q, par)
    i.is_reflected = 0

    plt.plot(*par.draw_locus(np.linspace(0, 10, 100)))
    plt.plot(*par.draw_tangent(q), c='k', ls='--', label='tangent')
    plt.plot(*par.draw_normal(q), ls='--', c='brown', label='normal')

    plt.plot([i_init, i.x_incident], [i.y, i.y], label='incident')
    i.is_reflected = 1
    
    x_incident_coord = list(i.get_ordinate(i.x_incident))
    i_init_coord = list(i.get_ordinate(i_init))
    plt.plot([x_incident_coord[0], i_init_coord[0]], [x_incident_coord[1], i_init_coord[1]], label='reflected')
    
    plt.hlines(0, -10, 20, ls='--', color='k')
    plt.scatter([a], [0], marker='*', color='r', s=100, label='Focus')
    plt.axis('scaled')
    plt.xlim(-10, 20)
    plt.ylim(-15, 15)
    plt.axis('off')
    plt.legend()
    
    plt.show()

In [5]:
interact(update_plot, q=(1, 12, 0.5), a=(2, 10, 1))

interactive(children=(FloatSlider(value=3.0, description='q', max=12.0, min=1.0, step=0.5), IntSlider(value=4,…

<function __main__.update_plot(q=3, a=4)>